In [5]:
import numpy as np
import pandas as pd
import pdpipe as pdp

import shapely
#import geopandas as gpd

import seaborn as sns
import folium

In [3]:
nyc_b_sub = pd.read_csv("./nyc_bikeshare_sub.csv")
nyc_b_sub_orig = nyc_b_sub
nyc_b_sub.head()

,Unnamed: 0,tripduration,starttime,stoptime,start_station_id,end_station_id,usertype,birthyear,gender
0,21679149,299,01-23-19 19:49:23,01-23-19 19:54:23,432.0,266.0,1.0,1991.0,1
1,9972891,3733,09-11-16 15:59:59,09-11-16 17:02:12,457.0,469.0,0.0,NaN,0
2,4105739,628,12-29-14 14:29:28,12-29-14 14:39:56,153.0,468.0,1.0,1959.0,1
3,9909812,594,09-07-16 19:10:26,09-07-16 19:20:20,326.0,537.0,1.0,1978.0,2
4,15688970,199,11-05-17 11:07:48,11-05-17 11:11:07,3061.0,3064.0,1.0,1990.0,1


In [7]:
pipe = pdp.DropNa(subset=["start_station_id","end_station_id"])
nyc_bsub = pipe(nyc_b_sub)

In [8]:
nyc_bsub.head()

,Unnamed: 0,tripduration,starttime,stoptime,start_station_id,end_station_id,usertype,birthyear,gender
0,21679149,299,01-23-19 19:49:23,01-23-19 19:54:23,432.0,266.0,1.0,1991.0,1
1,9972891,3733,09-11-16 15:59:59,09-11-16 17:02:12,457.0,469.0,0.0,NaN,0
2,4105739,628,12-29-14 14:29:28,12-29-14 14:39:56,153.0,468.0,1.0,1959.0,1
3,9909812,594,09-07-16 19:10:26,09-07-16 19:20:20,326.0,537.0,1.0,1978.0,2
4,15688970,199,11-05-17 11:07:48,11-05-17 11:11:07,3061.0,3064.0,1.0,1990.0,1
